
# 205229118

# Mahalakshmi S

# Lab15. Transfer Learning using CNN

### 1.import neccesary modules 


In [2]:
import datetime
import keras
import pandas as pd
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

### 2. Initialize some parameters

In [3]:
now = datetime.datetime.now  #get current time

batch_size = 128
num_classes = 5
epochs = 5

img_rows, img_cols = 28, 28
filters = 32
pool_size = 2
kernel_size = 3

### 3. Partition MNIST dataset

In [4]:
# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [5]:
# create two datasets: one with digits below 5 and one with 5 and above 
x_train_lt5 = x_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
x_test_lt5 = x_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

In [6]:
x_train_gte5 = x_train[y_train >= 5]
y_train_gte5 = y_train[y_train >= 5]-5
x_test_gte5 = x_test[y_test >= 5]
y_test_gte5 = y_test[y_test >= 5]-5

### 4. Define the 'feature' layers

In [7]:
INPUT_SHAPE = (img_rows,img_cols,1)

feature_layers=[Conv2D(filters=filters,kernel_size=kernel_size,activation='relu',input_shape=INPUT_SHAPE),
                Conv2D(filters=filters,kernel_size=kernel_size,activation='relu',input_shape=INPUT_SHAPE),
                MaxPooling2D(pool_size=pool_size),
                Dropout(0.25),
                Flatten()]

### 5. Define the "classification" layers

In [8]:
classification_layers = [Dense(128,activation='relu'),
                         Dropout(0.5),
                         Dense(5,activation='softmax')]

### 6. Define a Sequential model

In [9]:
model3 = Sequential(feature_layers+classification_layers)
model3.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 4608)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               589952    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0

### 7. Create a function 

In [10]:
def train_model3(model3,train,test,num_classes):
    
    train[0] = train[0].reshape(train[0].shape[0],img_rows,img_cols,1)
    test[0] = test[0].reshape(test[0].shape[0],img_rows,img_cols,1)

    train[0] = train[0].astype('float32')
    test[0] = test[0].astype('float32')
  
    train[0] /= 255
    test[0] /= 255

    print(train[0].shape)
    print(test[0].shape)

    s = pd.Series(train[1])
    train[1] = pd.get_dummies(s)
    train[1] = train[1].values

    s = pd.Series(test[1])
    test[1] = pd.get_dummies(s)
    test[1] = test[1].values


    model3.compile(optimizer='adadelta',loss='categorical_crossentropy',metrics=['accuracy'])

    train_start_time=now()
 
    model3.fit(train[0],train[1],epochs=epochs,verbose=2)
  
    print('\n')
    print('--------------------------------------------------------------------------------------')
    print('\n')
  
    print('Training time: %s' % (now() - train_start_time))
  
    score3=model3.evaluate(test[0],test[1],verbose=0)
  
    print('test loss ',score3[0])
    print('test accuracy ',score3[1])

### 8. Train your model on the digits 5,6,7,8,9

In [11]:
train_model3(model3,[x_train_gte5,y_train_gte5],[x_test_gte5,y_test_gte5],num_classes)

(29404, 28, 28, 1)
(4861, 28, 28, 1)
Epoch 1/5
919/919 - 5s - loss: 1.5677 - accuracy: 0.2927
Epoch 2/5
919/919 - 3s - loss: 1.4768 - accuracy: 0.4683
Epoch 3/5
919/919 - 3s - loss: 1.3637 - accuracy: 0.6124
Epoch 4/5
919/919 - 3s - loss: 1.2203 - accuracy: 0.6936
Epoch 5/5
919/919 - 3s - loss: 1.0671 - accuracy: 0.7399


--------------------------------------------------------------------------------------


Training time: 0:00:18.816339
test loss  0.908903956413269
test accuracy  0.8668997883796692


### 9. Freeze Feature Layers

In [12]:
for l in feature_layers:
    l.trainable = False

### 10. Print model summary()

In [13]:
model3.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 4608)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               589952    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0

### 11. Now, let's train our model on the digits 0, 1, 2, 3, 4

In [14]:
train_model3(model3,[x_train_lt5, y_train_lt5],[x_test_lt5, y_test_lt5], num_classes)

(30596, 28, 28, 1)
(5139, 28, 28, 1)
Epoch 1/5
957/957 - 3s - loss: 1.4488 - accuracy: 0.4064
Epoch 2/5
957/957 - 3s - loss: 1.2119 - accuracy: 0.5590
Epoch 3/5
957/957 - 3s - loss: 1.0322 - accuracy: 0.6965
Epoch 4/5
957/957 - 2s - loss: 0.8889 - accuracy: 0.7819
Epoch 5/5
957/957 - 2s - loss: 0.7749 - accuracy: 0.8230


--------------------------------------------------------------------------------------


Training time: 0:00:13.031757
test loss  0.6350380778312683
test accuracy  0.894726574420929


### 12. Now write code to reverse this training process

In [15]:
INPUT_SHAPE = (img_rows,img_cols,1)

feature_layers=[Conv2D(filters=filters,kernel_size=kernel_size,activation='relu',input_shape=INPUT_SHAPE),
                Conv2D(filters=filters,kernel_size=kernel_size,activation='relu',input_shape=INPUT_SHAPE),
                MaxPooling2D(pool_size=pool_size),
                Dropout(0.25),
                Flatten()]

classification_layers = [Dense(128,activation='relu'),
                         Dropout(0.5),
                         Dense(5,activation='softmax')]               

In [16]:
modelRev = Sequential(feature_layers+classification_layers)
modelRev.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 12, 12, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4608)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               589952    
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)              

In [18]:
train_model3(modelRev,[x_train_lt5,y_train_lt5],[x_test_lt5,y_test_lt5],num_classes)

(30596, 28, 28, 1)
(5139, 28, 28, 1)
Epoch 1/5
957/957 - 4s - loss: 1.5755 - accuracy: 0.2700
Epoch 2/5
957/957 - 3s - loss: 1.4787 - accuracy: 0.4764
Epoch 3/5
957/957 - 4s - loss: 1.3492 - accuracy: 0.6529
Epoch 4/5
957/957 - 4s - loss: 1.1697 - accuracy: 0.7611
Epoch 5/5
957/957 - 3s - loss: 0.9536 - accuracy: 0.8211


--------------------------------------------------------------------------------------


Training time: 0:00:18.180520
test loss  0.7526158094406128
test accuracy  0.9320879578590393


In [19]:
for l in feature_layers:
    l.trainable = False

In [20]:
modelRev.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 12, 12, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4608)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               589952    
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)              

In [21]:
train_model3(modelRev,[x_train_gte5,y_train_gte5],[x_test_gte5,y_test_gte5],num_classes)

(29404, 28, 28, 1)
(4861, 28, 28, 1)
Epoch 1/5
919/919 - 3s - loss: 1.6166 - accuracy: 0.3146
Epoch 2/5
919/919 - 2s - loss: 1.4332 - accuracy: 0.4070
Epoch 3/5
919/919 - 2s - loss: 1.2827 - accuracy: 0.5209
Epoch 4/5
919/919 - 2s - loss: 1.1578 - accuracy: 0.6119
Epoch 5/5
919/919 - 2s - loss: 1.0506 - accuracy: 0.6763


--------------------------------------------------------------------------------------


Training time: 0:00:12.510725
test loss  0.9183579087257385
test accuracy  0.7850236296653748


# Digits 5-9 is trained then last layer of network trained on Digit 0-4

## Digit 5-9   : Training time: 0:00:18.816339       

test loss  0.908903956413269

test accuracy  0.8668997883796692

## Digit 0-4   :  Training time: 0:00:13.031757

test loss  0.6350380778312683

test accuracy  0.894726574420929

# Digits 0-4 is trained then last layer of network trained on Digit 5-9

## Digit 0-4: Training time: 0:00:18.180520
test loss  0.7526158094406128

test accuracy  0.9320879578590393


## Digit 5-9:Training time: 0:00:12.510725
test loss  0.9183579087257385

test accuracy  0.7850236296653748